In [ ]:
from json import *
from copy import *
from heapq import *

#A state is a tuple of hallway, rooms, cost past
#The hallway is a list of chars of occupants or whitespaces for empty
#Rooms are an hallway index, a type of occupants and a list of the actual occupants
#Energy is the amount of energy to reach this state
#Past (if any) is a reference to the state that was unfolded to this

costs = {
    'A':1,
    'B':10,
    'C':100,
    'D':1000    
}

def heurestic(state):
    hallway, rooms, energy, past = state
    
    est = 0
    
    home = {}
    for i in range(len(rooms)):
        ix,tp,lst = rooms[i]
        home[tp]= rooms[i]

    #for every room 
    for ri in range(len(rooms)):
        ix,tp,lst = rooms[ri]

        #walkthrough list
        for li in range(len(lst)):            

            #if its a stranger
            if lst[li] != tp:
                #add base cost of travelling between rooms
                ix2,tp2,lst2 = home[lst[li]]
                est += (abs(ix - ix2) + 2) * costs[lst[li]]

    #scan hallway
    for i in range(len(hallway)):
        a = hallway[i]
        #find distance to every home + one step in 
        if a != ' ':
            ix,tp,lst = home[a]
            est += (abs(i - ix) + 1) * costs[a]

    
    #Todo add cost if someone is blocking
    #todo add cost of moving others in queue

    return est 

def room_has_strangers(lst, tp):
    for a in lst:
        if a != tp:
            return True
    return False

def isgoal(state):
    hallway, rooms, energy, past = state
    for x in hallway:
        if x != ' ':
            return False
    
    for ix,tp,lst in rooms:
        if room_has_strangers(lst, tp):
            return False
    return True


def state_parse(text, debug=False):
    if debug:
        print(text)
        print()
    lines = text.split("\n")
    hallway = [" " for x in lines[1].replace("#", "")]
    M = [list(l) for l in lines]
    rooms = []
    for x in range(len(lines[2])):
        if M[2][x] != "#":
            r = []
            for y in range(2):
                a = M[2+y][x]
                if a in "ABCD":
                    r.append(a)
            rooms.append([x-1,chr(ord('A')+len(rooms)),r])
    
    energy = 0
    past = None
    return hallway, rooms, energy,  past


#print(state_parse(open("i23_test0.txt").read()))
#print(state_parse(open("i23_test1.txt").read()))
#print(state_parse(open("i23_test2.txt").read()))
#print(state_parse(open("i23_test3.txt").read()))
#print(state_parse(open("i23_test4.txt").read()))
#print(state_parse(open("i23_test5.txt").read()))
#print(state_parse(open("i23_test6.txt").read()))
#print(state_parse(open("i23.txt").read()))

def state_print(state):
    hallway, rooms, energy, past = state
    
    queue_x = len(hallway)
    for x,h,s in rooms:
        queue_x = min(queue_x, x)
        
    for y in range(3 + 2):
        if y == 1:
            #hallway
            l = ["#"] + list(hallway) + ["#"]
        else:
            #other lines
            l = ["#"]*(len(hallway)+2)
            if y in [2,3]:
                for x,h,s in rooms:
                    ix = y-2
                    if len(s)> ix:
                        l[1+x] = s[ix]
                    else:
                        l[1+x]= " "
        #cut edges
        if y > 2:
            #x,h,s = rooms[0]
            x = queue_x
            l[:x] = l[-x:] = [" "] * x
        
        print("".join(l))


#state_print(state_parse(open("i23_test0.txt").read()))
#state_print(state_parse(open("i23_test1.txt").read()))
#state_print(state_parse(open("i23_test2.txt").read()))
#state_print(state_parse(open("i23_test3.txt").read()))
#state_print(state_parse(open("i23_test4.txt").read()))
#state_print(state_parse(open("i23_test5.txt").read()))



def state_unfold(state):
    newstates = []
    hallway, rooms, energy, past = state
    #go through all move options and spawn new situations
    
    #pickup room info for hallway moves
    home = {}
    
    #check every room if someone can exit, and build table of rooms
    for i in range(len(rooms)):
        ix, tp, lst = rooms[i]
        home[ix]=(i, rooms[i])
        
        if hallway[ix] == ' ' and room_has_strangers(lst, tp):
            hallway2, rooms2, energy2, past = deepcopy(state)
            for tp in rooms2[i][2]:
                energy2 += costs[tp]
            hallway2[ix] = rooms2[i][2].pop(0)
            newstates.append((hallway2, rooms2, energy2, state))

    #Sideways movement in hallway
    hr = range(len(hallway))
    for a in hr:
        for b in hr:
            if a != b:
                #target is open
                if hallway[b] == ' ' :
                    #source has passenger
                    if hallway[a] != ' ' :
                        dx = min(max(b-a,-1), 1)
                        path_is_clear = True
                        t = a + dx
                        while t != b:
                            if hallway[t] != ' ':
                                path_is_clear = False
                                break
                            t+=dx                        
                        if path_is_clear:                            
                            #dont stop in front of a home - unless its your type and it has no strangers 
                            #(actuall if so jump in directly)
                            if b in home.keys():
                                i, (ix, tp, lst) = home[b]
                                if not room_has_strangers(lst, tp):
                                    #room has no strangers
                                    if hallway[a] == tp:
                                        #and it is our type - jump in directly
                                        
                                        hallway2, rooms2, energy2, past = deepcopy(state)                                        
                                        rooms2[i][2].insert(0, hallway2[a])
                                        hallway2[a] = ' '
                                        
                                        #energy from moving to door
                                        energy2 += costs[tp]*abs(a-b)
                                        
                                        #energy from entering and moving others 
                                        for tp in rooms2[i][2]:
                                            energy2 += costs[tp]                                        
                                        
                                        newstates.append((hallway2, rooms2, energy2, state))
                                        
                            else:
                                #anyone can move here its not an exit
                                hallway2, rooms2, energy2, past = deepcopy(state)
                                hallway2[b] = hallway2[a]
                                energy2 += costs[hallway2[a]]*abs(a-b)
                                hallway2[a] = ' '
                                newstates.append((hallway2, rooms2, energy2, state))
    
    
    
    return newstates

    
def state_playback(state):
    l = []
    while True:
        l.append(state)
        if state[-1] is None:
            break
        else:
            state = state[-1]
    
    l.reverse()
    for state in l:
        hallway, rooms, energy, past = state
        state_print(state)

def state_hash(state):
    hallway, rooms, energy, past = state
    r = "".join(hallway)+","+",".join(["".join(lst) for ix,tp,lst in rooms])
    return r

    
def solve(fn, facit=None, playback=None):
    print("solving %s"%(fn))
    text = open(fn).read()
    start_state = state_parse(text)
    
    state_print(start_state)
    
    evaluate = []
    evaluate.append((heurestic(start_state)+start_state[2], start_state))
    visited = {}
    i = 0
    
    last_heurestic = None
    
    while len(evaluate):
        
        srt, state = heappop(evaluate)
        
        hallway, rooms, energy, past = state
        
        if i % 10000 == 0:
            print("%s step:%d queue:%d visited:%d cost:%d heurestic:%d"%(fn, i, len(evaluate), len(visited.keys()), energy, heurestic(state)))
        
        if isgoal(state):
            print("goal!!! iterations:%d energy:%d"%(i, energy))
            pb = False
            if facit is None:
                pass
            else:
                if energy == facit:
                    print("OK")
                else:
                    print("Energy is not expected")
                    pb = True            
            if pb or playback:
                print("="*20)
                print("PLAYBACK!")
                state_playback(state)
                print("="*20)
                print()            
            return energy
        
        #note that we visited this state
        visited[state_hash(state)] = 1        
        
        #unfold state to new states
        newstates = state_unfold(state)
        
        for new_state in newstates:
            if not state_hash(new_state) in visited:
                heappush(evaluate, (heurestic(new_state)+new_state[2], new_state))        
        i+=1
        
    print("No solution found")
    return None
    
#solve("i23_test0.txt",    46, True)
#solve("i23_test1.txt",   112, True)
#solve("i23_test2.txt",   810, True)
#solve("i23_test3.txt",   719, True)
solve("i23_test4.txt",  8470, True)
#solve("i23_test5.txt", 12521, True)
#solve("i23.txt",           1, True)

print("fin")


solving i23_test4.txt
#############
#           #
###D#C#B#A###
  #A#B#C#D#  
  #########  
i23_test4.txt step:0 queue:0 visited:0 cost:0 heurestic:8448
i23_test4.txt step:10000 queue:33280 visited:353 cost:1851 heurestic:7528
i23_test4.txt step:20000 queue:53503 visited:438 cost:680 heurestic:8778
i23_test4.txt step:30000 queue:88550 visited:662 cost:2240 heurestic:7323
i23_test4.txt step:40000 queue:126684 visited:793 cost:2240 heurestic:7343
i23_test4.txt step:50000 queue:161417 visited:869 cost:1163 heurestic:8449
i23_test4.txt step:60000 queue:197344 visited:975 cost:1259 heurestic:8363
i23_test4.txt step:70000 queue:236740 visited:1012 cost:1278 heurestic:8348
i23_test4.txt step:80000 queue:269894 visited:1053 cost:2283 heurestic:7348
i23_test4.txt step:90000 queue:293347 visited:1089 cost:2179 heurestic:7458
i23_test4.txt step:100000 queue:332371 visited:1133 cost:2285 heurestic:7354
i23_test4.txt step:110000 queue:367822 visited:1143 cost:1288 heurestic:8352
i23_test4.txt step: